<a name="POS-1"></a>

In [ ]:
from gensim.models import Word2Vec
import re

def return_documents():
    """
    Returns a list of documents, where each document is a string
    """
    from sklearn.datasets import fetch_20newsgroups
    dataset = fetch_20newsgroups()
    corpus = dataset.data
    return corpus

def tokenize_and_tag_documents(documents, nlp, sep_char="||||"):
    """
    Returns a list of lists of tokens. Each token has been 
    concatenated with its part of speech
    """
    for doc in nlp.pipe(documents, parse=False, entity=False):
        yield [word.lower_ + sep_char + word.pos_ for word in doc]
        

def build_model(tokenized_docs):
    """
    Returns a gensim Word2Vec model trained on our corpus
    """
    return Word2Vec(list(tokenized_docs), size=128)


documents = return_documents()
tokenized_and_tagged_documents = tokenize_and_tag_documents(documents, nlp)
model = build_model(tokenized_and_tagged_documents)


def return_alts(word):
    pattern = re.compile("^" + word + "\|\|\|\|")
    return list(filter(lambda x: bool(pattern.match(x)), model.wv.vocab.keys()))

if model:
    print(model.most_similar('real||||ADJ'))
    print()
    print(model.most_similar('real||||ADV'))

<a href="#POS-1">aaa</a>